# Initial setting 

## DeviceSetup

In [2]:
from laboneq.simple import SHFQC
from laboneq.dsl.device import DeviceSetup, create_connection
import numpy as np
from copy import deepcopy

host = "10.172.11.217"
#instantiate ad device setup object
setup = DeviceSetup(uid="test_setup")
#add a dataserver
setup.add_dataserver(host=host, port="8004",uid="test_server")

setup.add_instruments(SHFQC(uid="test_shfqc",server_uid="test_server", address="DEV12576", device_options="SHFQC/PLUS/QC6CH"))

setup.instruments

[SHFQC(
│   uid='test_shfqc',
│   interface='1GbE',
│   connections=[],
│   server_uid='test_server',
│   address='DEV12576',
│   device_options='SHFQC/PLUS/QC6CH',
│   reference_clock_source=None,
│   device_class=0
)
]

In [3]:
for ch_idx, q_idx in enumerate([15,16,17,18,19]):# enumerate 안에 len(list) 강제하고싶음
    #QA
    setup.add_connections(
        "test_shfqc",
        create_connection(to_signal=f"q{q_idx}/measure", ports="QACHANNELS/0/OUTPUT")) #signal_type="iq"
    setup.add_connections(
        "test_shfqc",
        create_connection(to_signal=f"q{q_idx}/acquire", ports="QACHANNELS/0/INPUT" )) #signal_type="acquire"
    #SG
    setup.add_connections(
        "test_shfqc",
        create_connection(to_signal=f"q{q_idx}/drive", ports=f"SGCHANNELS/{ch_idx}/OUTPUT")) #signal_type="iq" only iq allowed for SG??
    setup.add_connections(
        "test_shfqc",
        create_connection(to_signal=f"q{q_idx}/drive_ef", ports=f"SGCHANNELS/{ch_idx}/OUTPUT")) #signal_type="iq" only iq allowed for SG??
    setup.add_connections(
        "test_shfqc",
        create_connection(to_signal=f"q{q_idx}/drive_cr", ports=f"SGCHANNELS/{ch_idx}/OUTPUT"))#signal_type="iq"


## Setting QuantumElement, QPU and QuantumPlaform(optional)
1. By using `from_device_setup` function  
    DeviceSetup object we made doesn't have Calibration
2. By using `load_qubit_parameters` function
    This way 
- load parameters from JSON

In [4]:
from qpu_types.transmon import TransmonQubit, TransmonQubitParameters
from helper import load_qubit_parameters, save_qubit_parameters
from operations import TransmonOperations
from laboneq.dsl.quantum.qpu import QPU, QuantumPlatform



_loaded_qubits = load_qubit_parameters(filename="latest") 
#qubits = TransmonQubit.from_device_setup(setup) # Also can be done with load_qubit_parameters 
qubits = list(_loaded_qubits.values())
qpu = QPU(quantum_elements=qubits, quantum_operations=TransmonOperations())
#quantum_platform = QuantumPlatform(setup=setup, qpu=qpu)

# qpu.topology.add_edge("bus", "q16", "q17")
# qpu.topology.add_edge("bus", "q17", "q18")
# qpu.topology.add_edge("bus", "q18", "q19")
# qpu.topology.plot()




from laboneq.simple import workflow
folder_store = workflow.logbook.FolderStore("./experiment_store") 
folder_store.activate()
#folder_store.deactivate()
workflow.logbook.LoggingStore().activate()
#workflow.logbook.LogbookStore().deactivate()


## Common parameters check

In [5]:
# for _, qubit in enumerate(qubits):
#     ##################LOCAL OSCILLATOR FREQUENCY AND CHANNEL POWER ###################################
#     qubit.parameters.drive_lo_frequency = 4.8e9
#     qubit.parameters.readout_lo_frequency = 7.0e9
#     qubit.parameters.readout_range_in =  0
#     qubit.parameters.readout_range_out = -25
#     qubit.parameters.drive_range = 0 
    
    
    
#     ############READOUT PARAMETERS###################################################################
#     qubit.parameters.readout_length - 2.0e-6
#     qubit.parameters.readout_pulse['function'] = 'GaussianSquare'
#     qubit.parameters.readout_pulse['sigma'] = 0.1
#     qubit.parameters.readout_pulse['width'] = None
#     qubit.parameters.readout_pulse['risefall_sigma_ratio'] = 3.0
#     qubit.parameters.readout_amplitude = 1.0  #WARNING! 이거 SWEEP 할때 오버라이딩이 안되고 여기서 SCALING 됨
    
#     qubit.parameters.reset_delay_length = 200e-6
#     ###########SPECTROSCOPY PARAMETERS###############################################################
#     # Pulsed qubit spectroscopy drive ,  
#     qubit.parameters.spectroscopy_pulse['function'] = 'GaussianSquare'
#     qubit.parameters.spectroscopy_length = 5.0e-6
#     qubit.parameters.spectroscopy_pulse['sigma'] = 0.2
#     qubit.parameters.spectroscopy_pulse['width'] = None
#     qubit.parameters.spectroscopy_pulse['risefall_sigma_ratio'] = 3.0
#     qubit.parameters.spectroscopy_amplitude=0.01 
    

    

# qubits[3].parameters.readout_amplitude = 0.5
# qubits[4].parameters.readout_amplitude = 0.5
# qubits[2].parameters.readout_amplitude = 0.5


In [6]:
from laboneq.simple import Session
session = Session(setup)
session.connect(ignore_version_mismatch=True)

[2025.08.22 17:11:17.194] INFO    Logging initialized from [Default inline config in laboneq.laboneq_logging] logdir is /home/kriss/JSAHN/qubit-experiment/laboneq_output/log
[2025.08.22 17:11:17.196] INFO    VERSION: laboneq 2.57.0
[2025.08.22 17:11:17.197] INFO    Connecting to data server at 10.172.11.217:8004
[2025.08.22 17:11:17.203] INFO    Connected to Zurich Instruments LabOne Data Server version 25.07.0.507 at 10.172.11.217:8004
[2025.08.22 17:11:17.224] INFO    Configuring the device setup
[2025.08.22 17:11:17.234] INFO    The device setup is configured


# 1 Spectroscopy

Problem on options
If option is given as `experiment_workflow.options()` we cannot change options for create_experiment.
To change options for experiment we have to give `ResonatorSpectroscopyExperimentOptions'

## 1.1 Resonator Spectroscopy
- Default `AcquisitioType.SPECTROSOCPY_IQ`

In [8]:
from laboneq.simple import AcquisitionType
from laboneq_applications.experiments import resonator_spectroscopy
q = qubits[3]
temporary_parameters = q.parameters.copy()
temporary_parameters.readout_amplitude = 0.1
temporary_parameters.readout_length = 2e-6
###############################################################
frequencies = q.parameters.readout_resonator_frequency + np.linspace(-5e6,+5e6,101)
################################################################
options = resonator_spectroscopy.experiment_workflow.options()
options.update(True)
#print(workflow.show_fields(options))

###################################################################
res_spec = resonator_spectroscopy.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    options=options,
    temporary_parameters={q.uid : temporary_parameters}
)
#res_spec_compiled = session.compile(resonator_spectroscopy.create_experiment(qpu=qpu,qubit=q,frequencies=frequencies,options=options))
res_spec_result = res_spec.run()
print(res_spec_result.tasks['analysis_workflow'].output)


[2025.08.22 17:12:48.536] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.22 17:12:48.536] INFO      Workflow 'resonator_spectroscopy': execution started at 2025-08-22            
[2025.08.22 17:12:48.537] INFO      08:12:48.536103Z                                                              
[2025.08.22 17:12:48.537] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.22 17:12:48.542] WARNING Type <class 'dict'> not supported by the serializer [name: resonator_spectroscopy.input.temporary_parameters].
[2025.08.22 17:12:48.552] INFO    Task 'temporary_qpu': started at 2025-08-22 08:12:48.552560Z
[2025.08.22 17:12:48.553] WARNING Type <class 'dict'> not supported by the serializer [name: temporary_qpu.input.temporary_parameters].
[2025.08.22 17:12:48.554] INFO    Task 'temporary_qpu': ended at 2025-08-22 08:12:48.553930Z
[2025.08.22 17:12:48.556] INFO    Task 'temporary_quantum_elements_from

## 1.1 + Resonator Spectroscopy Amplitude

In [ ]:
from laboneq.simple import AcquisitionType
from laboneq_applications.experiments import resonator_spectroscopy_amplitude
q = qubits[3]
temporary_parameters = q.parameters.copy()
temporary_parameters.readout_amplitude = 1.0
temporary_parameters.readout_length = 2e-6
###############################################################
frequencies = q.parameters.readout_resonator_frequency + np.linspace(-2e6,+2e6,101)
amplitudes = np.linspace(0,1.0,11)
################################################################
options = resonator_spectroscopy.experiment_workflow.options()
#print(workflow.show_fields(options))

###################################################################
res_spec_amp = resonator_spectroscopy_amplitude.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    amplitudes=amplitudes,
    options=options,
    temporary_parameters={q.uid : temporary_parameters}
)
#res_spec_amp_compiled = session.compile(resonator_spectroscopy.create_experiment(qpu=qpu,qubit=q,frequencies=frequencies,options=options))
res_spec_amp_result = res_spec_amp.run()


In [ ]:
from copy import deepcopy
from laboneq_applications.experiments import qubit_spectroscopy
q = qubits[3]
temporary_parameters = q.parameters.copy()
temporary_parameters.spectroscopy_amplitude = 0.001
temporary_parameters.spectroscopy_length = 30e-6

q.spectroscopy_parameters()

temporary_parameters.spectroscopy_length

## 1.2 Qubit Spectroscopy

In [10]:
from copy import deepcopy
from laboneq_applications.experiments import qubit_spectroscopy
q = qubits[3]
temporary_parameters = q.parameters.copy()
temporary_parameters.spectroscopy_amplitude = 0.001
temporary_parameters.spectroscopy_length = 30e-6
###############################################################
frequencies = q.parameters.resonance_frequency_ge +np.linspace(-2e6,2e6,101)
################################################################
options = qubit_spectroscopy.experiment_workflow.options()
options.update(True)


#print(workflow.show_fields(options))

###################################################################
qubit_spec = qubit_spectroscopy.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    frequencies=frequencies,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

qubit_spec_result = qubit_spec.run()
#qubit_spec_compiled = session.compile(qubit_spectroscopy.create_experiment(qpu=qpu,qubit=q,frequencies=frequencies, options=options))
qubit_parameters = qubit_spec_result.tasks['analysis_workflow'].output
print(qubit_parameters)


[2025.08.22 17:13:48.224] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.22 17:13:48.224] INFO      Workflow 'qubit_spectroscopy': execution started at 2025-08-22                
[2025.08.22 17:13:48.225] INFO      08:13:48.223948Z                                                              
[2025.08.22 17:13:48.225] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.22 17:13:48.228] WARNING Type <class 'dict'> not supported by the serializer [name: qubit_spectroscopy.input.temporary_parameters].
[2025.08.22 17:13:48.236] INFO    Task 'temporary_qpu': started at 2025-08-22 08:13:48.236189Z
[2025.08.22 17:13:48.236] WARNING Type <class 'dict'> not supported by the serializer [name: temporary_qpu.input.temporary_parameters].
[2025.08.22 17:13:48.237] INFO    Task 'temporary_qpu': ended at 2025-08-22 08:13:48.237625Z
[2025.08.22 17:13:48.240] INFO    Task 'temporary_quantum_elements_from_qpu

## 1.2 + Qubit Spectroscopy Amplitude

# 2. Rabi Amplitude

In [12]:
from laboneq_applications.experiments import amplitude_rabi
q = qubits[3]
temporary_parameters = q.parameters.copy()
temporary_parameters.ge_drive_amplitude_pi=1.0
#######################################################################
amplitudes = np.linspace(0,1.0,51)
#######################################################################
options = amplitude_rabi.experiment_workflow.options()
options.update(True)
options.use_cal_traces(False)
#print(workflow.show_fields(options))

###################################################################
rabi_amp = amplitude_rabi.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    amplitudes=amplitudes,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

rabi_amp_result = rabi_amp.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(rabi_amp_result.tasks['analysis_workflow'].output)

[2025.08.22 17:15:02.888] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.22 17:15:02.889] INFO      Workflow 'amplitude_rabi': execution started at 2025-08-22 08:15:02.888298Z   
[2025.08.22 17:15:02.889] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.22 17:15:02.894] WARNING Type <class 'dict'> not supported by the serializer [name: amplitude_rabi.input.temporary_parameters].
[2025.08.22 17:15:02.905] INFO    Task 'temporary_qpu': started at 2025-08-22 08:15:02.904804Z
[2025.08.22 17:15:02.905] WARNING Type <class 'dict'> not supported by the serializer [name: temporary_qpu.input.temporary_parameters].
[2025.08.22 17:15:02.906] INFO    Task 'temporary_qpu': ended at 2025-08-22 08:15:02.906259Z
[2025.08.22 17:15:02.908] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-08-22 
[2025.08.22 17:15:02.908] INFO    08:15:02.908338Z
[2025.08.22 17:15:02.909] INFO    Task 'te

# 3. Ramsey

# WARNING 
현재 자동으로 q17_readout_acquire_osc' on signal '/logical_signal_groups/q17/acquire' to SOFTWARE 로 설정되는거 어디서 하는지 확인해야함 , cr_Drive도 AUTO인데 뭐지.....

하여튼 HARDWARE여야함

In [13]:
from laboneq_applications.experiments import ramsey
q = qubits[3]
temporary_parameters = q.parameters.copy()
#######################################################################
delays = np.linspace(0,1e-6,10)
detunings = 0.2e6
#######################################################################
# options = ramsey.experiment_workflow.options()
# options.transition("ge")
# options.cal_states("ge")
# options.update(True)
#print(workflow.show_fields(options))

###################################################################
ramsey_freq = ramsey.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    delays=delays,
    detunings=detunings,
    #temporary_parameters={q.uid : temporary_parameters},
    #options=options
)

ramsey_freq_result = ramsey_freq.run()
#ramsey_compiled = session.compile(ramsey.create_experiment(qpu=qpu,qubit=q,delays=delays,detunings=detunings, options=options))
print(ramsey_freq_result.tasks['analysis_workflow'].output)

[2025.08.22 17:15:37.080] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.22 17:15:37.080] INFO      Workflow 'ramsey': execution started at 2025-08-22 08:15:37.079896Z           
[2025.08.22 17:15:37.080] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.22 17:15:37.086] INFO    Task 'temporary_qpu': started at 2025-08-22 08:15:37.086645Z
[2025.08.22 17:15:37.088] INFO    Task 'temporary_qpu': ended at 2025-08-22 08:15:37.088093Z
[2025.08.22 17:15:37.092] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-08-22 
[2025.08.22 17:15:37.093] INFO    08:15:37.092106Z
[2025.08.22 17:15:37.094] INFO    Task 'temporary_quantum_elements_from_qpu': ended at 2025-08-22 08:15:37.094313Z
[2025.08.22 17:15:37.098] INFO    Task 'create_experiment': started at 2025-08-22 08:15:37.097917Z
[2025.08.22 17:15:37.100] INFO    Task 'create_experiment': ended at 2025-08-22 08:15:37.100375Z
[202


thread '<unnamed>' panicked at src/rust/codegenerator/src/passes/handle_signatures.rs:23:9:
cannot increment oscillator phase w/o command table
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


PanicException: cannot increment oscillator phase w/o command table

# Error Amplification

In [15]:
from laboneq_applications.experiments import amplitude_fine
q = qubits[3]
temporary_parameters = q.parameters.copy()

#######################################################################
repetitions =np.arange(1,50)
#######################################################################
options = amplitude_fine.experiment_workflow_x180.options()
options.update(True)
options.use_cal_traces(True)
#print(workflow.show_fields(options))

###################################################################
error_amp = amplitude_fine.experiment_workflow_x180(
    session=session,
    qpu=qpu,
    qubits=q,
    repetitions=repetitions,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

error_amp_result = error_amp.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(error_amp_result.tasks['analysis_workflow'].output)

[2025.08.22 17:16:28.384] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.22 17:16:28.385] INFO      Workflow 'experiment_workflow_x180': execution started at 2025-08-22          
[2025.08.22 17:16:28.385] INFO      08:16:28.384332Z                                                              
[2025.08.22 17:16:28.386] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.22 17:16:28.390] WARNING Type <class 'dict'> not supported by the serializer [name: experiment_workflow_x180.input.temporary_parameters].
[2025.08.22 17:16:28.398] INFO    Task 'temporary_qpu': started at 2025-08-22 08:16:28.398421Z
[2025.08.22 17:16:28.399] WARNING Type <class 'dict'> not supported by the serializer [name: temporary_qpu.input.temporary_parameters].
[2025.08.22 17:16:28.400] INFO    Task 'temporary_qpu': ended at 2025-08-22 08:16:28.400164Z
[2025.08.22 17:16:28.403] INFO    Task 'temporary_quantum_elements_fr

In [17]:
from laboneq_applications.experiments import amplitude_fine
q = qubits[3]
temporary_parameters = q.parameters.copy()

#######################################################################
repetitions =np.arange(1,50)
#######################################################################
options = amplitude_fine.experiment_workflow_x90.options()
options.update(True)
options.use_cal_traces(True)
#print(workflow.show_fields(options))

###################################################################
error_amp_half = amplitude_fine.experiment_workflow_x90(
    session=session,
    qpu=qpu,
    qubits=q,
    repetitions=repetitions,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

error_amp_half_result = error_amp_half.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(error_amp_half_result.tasks['analysis_workflow'].output)

[2025.08.22 17:17:53.280] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.22 17:17:53.281] INFO      Workflow 'experiment_workflow_x90': execution started at 2025-08-22           
[2025.08.22 17:17:53.281] INFO      08:17:53.280657Z                                                              
[2025.08.22 17:17:53.281] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.22 17:17:53.284] WARNING Type <class 'dict'> not supported by the serializer [name: experiment_workflow_x90.input.temporary_parameters].
[2025.08.22 17:17:53.292] INFO    Task 'temporary_qpu': started at 2025-08-22 08:17:53.292674Z
[2025.08.22 17:17:53.293] WARNING Type <class 'dict'> not supported by the serializer [name: temporary_qpu.input.temporary_parameters].
[2025.08.22 17:17:53.294] INFO    Task 'temporary_qpu': ended at 2025-08-22 08:17:53.293839Z
[2025.08.22 17:17:53.296] INFO    Task 'temporary_quantum_elements_fro

In [28]:
from laboneq_applications.experiments import drag_q_scaling
q = qubits[2]
temporary_parameters = q.parameters.copy()

#######################################################################
q_scalings=np.linspace(-0.01,+0.01,51)
#######################################################################
options = drag_q_scaling.experiment_workflow.options()
options.update(True)
#options.use_cal_traces(True)
#print(workflow.show_fields(options))

###################################################################
drag = drag_q_scaling.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    q_scalings=q_scalings,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

drag_result = drag.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(drag_result.tasks['analysis_workflow'].output)

[2025.08.22 17:30:16.373] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.22 17:30:16.373] INFO      Workflow 'drag_q_scaling': execution started at 2025-08-22 08:30:16.372851Z   
[2025.08.22 17:30:16.374] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.22 17:30:16.377] WARNING Type <class 'dict'> not supported by the serializer [name: drag_q_scaling.input.temporary_parameters].
[2025.08.22 17:30:16.385] INFO    Task 'temporary_qpu': started at 2025-08-22 08:30:16.385570Z
[2025.08.22 17:30:16.386] WARNING Type <class 'dict'> not supported by the serializer [name: temporary_qpu.input.temporary_parameters].
[2025.08.22 17:30:16.387] INFO    Task 'temporary_qpu': ended at 2025-08-22 08:30:16.386946Z
[2025.08.22 17:30:16.390] INFO    Task 'temporary_quantum_elements_from_qpu': started at 2025-08-22 
[2025.08.22 17:30:16.390] INFO    08:30:16.390179Z
[2025.08.22 17:30:16.391] INFO    Task 'te

# Dispersive shift

In [19]:
from laboneq_applications.experiments import dispersive_shift
q = qubits[3]
temporary_parameters = q.parameters.copy()

#######################################################################
frequencies=q.parameters.readout_resonator_frequency + np.linspace(-5e6,+5e6,101)
states=["g","e"]

#######################################################################
options = dispersive_shift.experiment_workflow.options()
options.update(True)
#options.use_cal_traces(True)
#print(workflow.show_fields(options))

###################################################################
dispersive = dispersive_shift.experiment_workflow(
    session=session,
    qpu=qpu,
    qubit=q,
    frequencies=frequencies,
    states=states,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

dispersive_result = dispersive.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))
print(dispersive_result.tasks['analysis_workflow'].output)

[2025.08.22 17:20:15.630] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.22 17:20:15.630] INFO      Workflow 'dispersive_shift': execution started at 2025-08-22                  
[2025.08.22 17:20:15.631] INFO      08:20:15.630039Z                                                              
[2025.08.22 17:20:15.632] INFO     ────────────────────────────────────────────────────────────────────────────── 
[2025.08.22 17:20:15.639] WARNING Type <class 'dict'> not supported by the serializer [name: dispersive_shift.input.temporary_parameters].
[2025.08.22 17:20:15.647] INFO    Task 'temporary_qpu': started at 2025-08-22 08:20:15.647629Z
[2025.08.22 17:20:15.648] WARNING Type <class 'dict'> not supported by the serializer [name: temporary_qpu.input.temporary_parameters].
[2025.08.22 17:20:15.649] INFO    Task 'temporary_qpu': ended at 2025-08-22 08:20:15.648942Z
[2025.08.22 17:20:15.651] INFO    Task 'temporary_quantum_elements_from_qpu':

# IQ Blobs

In [ ]:
from laboneq_applications.experiments import iq_blobs
q = qubits[2]
temporary_parameters = q.parameters.copy()

#######################################################################

states=["g","e"]

#######################################################################
options = iq_blobs.experiment_workflow.options()
#options.update(False)
#options.use_cal_traces(True)
#print(workflow.show_fields(options))

###################################################################
iq = iq_blobs.experiment_workflow(
    session=session,
    qpu=qpu,
    qubits=q,
    states=states,
    temporary_parameters={q.uid : temporary_parameters},
    options=options
)

iq_result = iq.run()
#qubit_spec_compiled = session.compile(amplitude_rabi.create_experiment(qpu=qpu,qubit=q,amplitudes=amplitudes, options=options))


# Save qubit

In [29]:
from helper import load_qubit_parameters, save_qubit_parameters
save_qubit_parameters(qubits={ qubit.uid : qubit for qubit in qubits}, save_folder="./qubit_parameters", filename="kaist")

Qubit parameters stored as JSON in /home/kriss/JSAHN/qubit-experiment/qubit_parameters/20250822-1731_kaist.json


In [ ]:
from experiments import direct_cr_hamiltonian_tomography
from experiments.direct_cr_hamiltonian_tomography import DirectCRHamiltonianTomographyOptions
############################################################
options = direct_cr_hamiltonian_tomography.experiment_workflow.options()
options.do_analysis(False)
options.update(False)
options.count(2048)
workflow.show_fields(options)